Developed by: Thomas.Reimann@tu-dresden.de / Sophie.Pfoertner@mailbox.tu-dresden.de / Anne.Pfoertner@mailbox.tu-dresden.de
<br>Last change: 2020 11 06
<br>Current state: funktional
<br>ToDo:

- Code optimieren ? (TR)
- Formatierung (Arrow / GWN Pfeile / Grundwasser Dreieck) in andere übertragen
- Bei Frage 2: Antwort über Textauswahl (Dropdown?)

<center>
    
### MWW01 - Grundwasserbewirtschaftung mit Computermodellen
### Thema 02: Konzeptionelle Modelle
# Physikalische und Hydraulische Randbedingungen  </center>
Dieses Notebook erläutert den Unterschied zwischen physikalischen und hydraulischen Randbedingungen anhand eines einfach Beispiels. Mit den zugehörigen Aufgaben kann das Verhalten der einzelnen Randbedingungen näher untersucht werden.

### Lernziele:
Nachdem Sie dieses Notebook erfolgreich bearbeitet haben, können Sie:
* Die Unterschiede zwischen hydraulischen und physikalischen Randbedingungen für Grundwasserströmungsmodelle erläutern;
* Disskutieren, mit welchen Möglichkeiten eine Grundwasserscheide für ein Grundwasserströmungsmodell berücksichtigt werden kann.

### Ausgangssituation
Betrachtet wird ein ungespannter, homogener und isotroper Grundwasserleiter, der durch zwei Standgewässer begrenzt wird. Der Grundwasserleiter wird zusätzlich durch Grundwasserneubildung gespeist. Die Grundwasserhydraulik kann durch eine 1D Vereinfachung der allgemeinen Grundwasserströmungsgleichung beschrieben werden, für die folgende analytische Lösung existiert (siehe nachfolgende Abbildung)

<br>
<center>$\large h(x)=\sqrt{H_l^2-\frac{H_l^2-H_r^2}{L}x+\frac{q_n}{K}x(L-x)}$</center>
<br>
mit

* $h$ = Potentialhöhe
* $x$ = Ortskoordinate entlang der Strömungsrichtung
* $K$ = hydraulische Leitfähigkeit
* $L$ = Distanz zwischen den beiden definierten Potentialhöhen
* $H_l$ = definierte Potentialhöhe am linken Rand
* $H_r$ = definierte Potentialhöhe am rechten Rand
* $q_n$ = Grundwasserneubildung

<figure>
  <img src="FIG/MWW01_T02_1D_unconfied_BC_4.png" alt="Trulli" style="width:50%">
  <figcaption>Abb.1 - Skizze der hydrogeologischen Situation.</figcaption>
</figure>

<br> Im nachfolgenden Teil des Notebooks wird die analytische Lösung ermittelt und grafisch dargestellt.

In [149]:
# Initialize librarys
from scipy.special import erfc, erf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from ipywidgets import *

In [150]:
# Definition of the function
def head(hl, hr, L, R, K,y_scale):
    x = np.arange(0, L,L/1000)
    R=R/1000/365.25/86400
    h=(hl**2-(hl**2-hr**2)/L*x+(R/K*x*(L-x)))**0.5
    
    # PLOT FIGURE
    fig = plt.figure(figsize=(9,6))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(x,h)
    ax.set(xlabel='x (m)', ylabel='Druckpotential (m)',title='Potentialhöhe für die 1D ungespannte Grundwasserströmung')
    ax.fill_between(x,0,h, facecolor='lightblue')
    
    # BOUNDARY CONDITIONS hl, hr
    ax.vlines(0, 0, hl, linewidth = 10, color='b')
    ax.vlines(L, 0, hr, linewidth = 10, color='b')
    
    # MAKE 'WATER'-TRIANGLE
    y_range = abs((hl*(1-y_scale/100))-(hr*(1+y_scale/100)))
    h_arrow = (hl**2-(hl**2-hr**2)/L*(L*0.96)+(R/K*(L*0.96)*(L-(L*0.96))))**0.5  #water level at arrow
    ax.arrow(L*0.96,(h_arrow+(h_arrow*0.004)), 0, -0.01, fc="k", ec="k", head_width=(L*0.015), head_length=(y_range*0.03))
    #ax.hlines(y= h_arrow-(h_arrow*0.0005), xmin=L*0.95, xmax=L*0.97, colors='blue')   
    ax.hlines(y= h_arrow-(y_range*0.01), xmin=L*0.95, xmax=L*0.97, colors='blue') 
    #ax.hlines(y= h_arrow-(h_arrow*0.001), xmin=L*0.955, xmax=L*0.965, colors='blue')
    ax.hlines(y= h_arrow-(y_range*0.015), xmin=L*0.955, xmax=L*0.965, colors='blue')

    #ARROWS FOR RECHARGE 
    if R != 0:
        head_length=(R*86400*365.25*1000*0.0005)*y_range
        h_rch1 = (hl**2-(hl**2-hr**2)/L*(L*0.25)+(R/K*(L*0.25)*(L-(L*0.25))))**0.5  #water level at arrow for Recharge Posotion 1
        ax.arrow(L*0.25,(h_rch1+head_length), 0, -0.01, fc="k", ec="k", head_width=(L*0.03), head_length=head_length)
        h_rch2 = (hl**2-(hl**2-hr**2)/L*(L*0.50)+(R/K*(L*0.50)*(L-(L*0.50))))**0.5  #water level at arrow for Recharge Postition 2
        ax.arrow(L*0.50,(h_rch2+head_length), 0, -0.01, fc="k", ec="k", head_width=(L*0.03), head_length=head_length)
        h_rch3 = (hl**2-(hl**2-hr**2)/L*(L*0.75)+(R/K*(L*0.75)*(L-(L*0.75))))**0.5  #water level at arrow for Recharge Position 3
        ax.arrow(L*0.75,(h_rch3+head_length), 0, -0.01, fc="k", ec="k", head_width=(L*0.03), head_length=head_length)

    
    #Grundwasserscheide
    max_y = max(h)
    max_x = x[h.argmax()]
    R_min_ms=K*abs(hl**2-hr**2)/L**2
    if R>R_min_ms:
        plt.vlines(max_x,0,max_y, color="r")

    plt.ylim(hl*(1-y_scale/100),hr*(1+y_scale/100))
    plt.xlim(-50,L+50)
    plt.text(L, (hr*(1+y_scale/100))-0.1*y_range, r'GWN: {:.3e} m/s '.format(R), horizontalalignment='right', bbox=dict(boxstyle="square", facecolor='azure'),fontsize=12)
    ax.grid()
    plt.show()
    #print('R: ',R, ' m/s')


In [151]:
# Computation

interact(head,
         y_scale = widgets.BoundedFloatText(value=5, min=1, max=100, step=1, description='Skal. y-Achse:', disabled=False),
         hl=widgets.BoundedFloatText(value=100, min=0, max=1000, step=1, description='H_l:', disabled=False),
         hr=widgets.BoundedFloatText(value=102, min=0, max=1000, step=1, description='H_r:', disabled=False),
         L= widgets.BoundedFloatText(value=2500,min=0, max=20000,step=100, description='L:' , disabled=False),
         R=widgets.FloatSlider(value=0,min=-500, max=500, step=10,description='GWN:', disabled=False),
         K=widgets.FloatLogSlider(value=0.0001,base=10,min=-6, max=-2, step=0.1,description='K:',readout=True,readout_format='.2e'))

interactive(children=(BoundedFloatText(value=100.0, description='H_l:', max=1000.0, step=1.0), BoundedFloatTex…

<function __main__.head(hl, hr, L, R, K, y_scale)>

### Anleitung zur eigenen Untersuchung

1) Stellen Sie mit Hilfe der Schieberegler die initialen Größen für Grundwasserneubildung (GWN) und hydraulische Leitfähigkeit K ein (Hinwweis: Zahlenwerte können direkt modifiziert werden)
* Hl = 100 m
* Hr = 102 m
* GWN = 200 mm/a und
* K = 2e-5.
<br>

**FRAGE 1:** Es bildet sich eine Grundwasserscheide aus (rote Linie). Beschreiben Sie, in welche Richtung sich das aus der Grundwasserneubildung zuströmende Wasser bewegt.
<br>
Die Situation soll mit einem numerischen Modell simuliert werden. Dazu werden drei mögliche Randbedingungen in Betracht gezogen, welche in Abbildung 2 mit **A, B und C** bezeichnet werden.
<br>

<figure>
  <img src="FIG/MWW01_T02_1D_unconfied_BC_2.png" alt="Abb. 2" style="width:50%">
  <figcaption>Abb.2 - Skizze der hydrogeologischen Situation mit weiteren Angaben zur Aufgabenstellung.</figcaption>
</figure>

**Frage 2:** Entscheiden Sie für die Randbedingungen **A, B, und C** (S. Abb. 2), ob es sich um physikalische oder hydraulische Randbedingungen handelt und notieren Sie Ihre (initiale) Antwort in der folgenden Box des Notebooks:

In [59]:
A = "Phy"
B = "HIER EINFACH ANTWORT NOTIEREN"
C = "HIER EINFACH ANTWORT NOTIEREN"

print("RB A: ",A,", RB B: ",B,", RB C:",C)

RB A:  Phy , RB B:  HIER EINFACH ANTWORT NOTIEREN , RB C: HIER EINFACH ANTWORT NOTIEREN


**Frage 3:** In einem relativ trockenem Jahr reduziert sich die Grundwasserneubildung auf 100 mm/a.
* Stellen Sie diesen Wert ein und beschreiben Sie, wie sich die Wasserscheide verhält.
* Welche Auswirkungen hat die reduzierte GWN für die Randbedingung **B** (siehe Abb. 2)? Diskutieren Sie kritisch!
* Abb. 2 zeigt ebenfalls zwei Modellvarianten (V1 und V2) - welche würden Sie für eine numerische Simulation empfehlen und warum?

**Frage 4:** Für die numerische Simulation stehen nur begrenzte Ressourcen zur Verfügung, so dass Modell **V1** mit der geringeren räumlichen Ausdehnung verwendet wird. Bei dieser Entscheidung wurde davon ausgegangen, dass sich die Grundwasserneubildung nicht ändert.
* Welche Art von Randbedingung schlagen Sie für **B** (siehe Abb. 2) vor? Notieren Sie Ihre (initiale) Antwort in der folgenden Box des Notebooks:

In [145]:
D = "HIER EINFACH RB NOTIEREN"
print("Vorschlag Randbedingung: ",D)

Vorschlag Randbedingung:  HIER EINFACH RB NOTIEREN


**Frage 5:** Infolge von Maßnahmen zur Wasserbewirtschaftung wird das Druckpotential in den Randbedingungen **A und B** jeweils um einen Meter erhöht.
* Führen Sie diese Änderungen in der analyischen Lösung durch. Beschreiben Sie die Änderung von Lage und Druckpotential der Grundwasserscheide.
* Prüfen Sie Ihre Entscheidung in Frage 4 hinsichtlich der Art der RB, revidieren Sie diese gegebenfalls und Begründen Sie ihre endgültige Auswahl.